
# 270. 컴퓨터 비전(Vision)을 위한 전이학습(Transfer Learning)
=======================================================

- 실제로 충분한 크기의 데이터셋을 갖추기는 상대적으로 드물기 때문에, (무작위 초기화를 통해) 처음부터 합성곱 신경망(Convolutional Network) 전체를 작성하는 경우는 매우 적다.   


- 대신, 매우 큰 데이터셋(예. 100가지 분류에 대해 120만개의 이미지가 포함된 ImageNet)에서 합성곱 신경망(ConvNet)을 미리 학습한 후, 이 합성곱 신경망을 관심있는 작업을 위한 초기 설정 또는 고정된 특징 추출기(fixed feature extractor)로 사용

### 전이학습의 2 가지 시나리오:

-  **합성곱 신경망의 미세조정(finetuning)**: 신경망을 ImageNet 등으로 미리 학습한 신경망으로 초기화하고 parameter 미세 조정  


-  **고정된 특징 추출기로써의 합성곱 신경망**: 마지막 완전 연결층을 제외한 모든 신경망의 가중치를 고정. 마지막의 완전 연결층은 새로운 무작위의 가중치를 갖는 계층으로 대체되어 이 계층만 학습.  


### torch 제공 pre-trained models
```
    import torchvision.models as models  
    
    resnet18 = models.resnet18()  
    alexnet = models.alexnet()  
    vgg16 = models.vgg16()  
    squeezenet = models.squeezenet1_0()  
    densenet = models.densenet161()  
    inception = models.inception_v3()  
    googlenet = models.googlenet()  
    shufflenet = models.shufflenet_v2_x1_0()  
    mobilenet = models.mobilenet_v2()  
    resnext50_32x4d = models.resnext50_32x4d()  
    wide_resnet50_2 = models.wide_resnet50_2()  
    mnasnet = models.mnasnet1_0()  
```   

In [ ]:
# 프로그램 실행 중 사용할 수 있는 임시 폴더 생성
# 블록 벗어나면 자동 삭제

데이터 불러오기
---------------

- **개미** 와 **벌** 을 분류하는 이진 분류 모델을 학습
    - 대략 120장 정도의 훈련 이미지와, 75장의 검증용 이미지를 이용
    - 전이학습을 통해 소량의 데이터로도 일반화 가능
    - ImageNet의 일부 data 이용

-  데이터를 [여기](https://download.pytorch.org/tutorial/hymenoptera_data.zip)
   에서 다운로드 받아 현재 디렉토리에 압축을 푼다

In [ ]:
# 데이터 증가(augmentation) 및 ResNet 에 적합한 normalization 적용

# ImageFolder 라이브러리

- 계층적인 폴더 구조를 가지고 있는 데이터셋을 불러올 때 사용할 수 있다. 각 이미지들이 자신의 레이블(Label) 이름으로 된 폴더 안에 들어가 있는 구조라면, ImageFolder 라이브러리를 이용하여 이를 바로 불러와 객체로 만들면 된다.

ImageFolder를 사용하기 위해선 가장 먼저 수집된 데이터의 폴더구조를 아래와 같이 설계해야한다  

최상위 경로 아래에 각각의 class name을 가지는 폴더를 구성하고 그 하위경로에 이미지가 저장되어있는 방식이다.
```
root_dir
    | --- ants/
    |      |-- 0001.jpg
    |      |-- 0002.jpg
    |      |-- ...
    | --- bees/
    |      |-- 0001.jpg
    |      |-- 0002.jpg
    |      |-- ...
    | --- rabbit/
    |      |--...
```

### `datasets.ImageFolder(folder, transform)` data loader 생성

### 일부 이미지 시각화하기

- `torchvision.utils.make_grid(tensor)`  
  - `tensor (Tensor 또는 list)`: 4차원 텐서 `(B x C x H x W)` 형태의 미니 배치 텐서 또는 같은 크기의 이미지 리스트입니다. (여기서 B는 이미지 개수, C는 채널 수, H와 W는 이미지의 높이와 너비)  
  - `nrow (int, 선택)`: 한 줄에 표시할 이미지 개수입니다. 최종 그리드는 `(B / nrow, nrow)` 형태가 됩니다. 기본값은 8입니다.

In [ ]:
def imshow(inp, title=None):
    # 입력 텐서 차원을 재배열. PyTorch - [C, H, W], matplotlib - [H, W, C]
    # 이미지를 정규화할 때 사용된 평균과 표준편차를 정의 - 모델이 학습될 때 사용된 값들
    # 정규화된 이미지 데이터를 원래의 범위로 되돌립니다.
    # np.clip 함수를 사용하여 이미지 데이터의 값이 0과 1 사이로 제한됩니다.
    # 이미지 위에 제목 표시

In [ ]:
# 훈련 데이터 로더에서 첫 번째 배치를 가져옵니다.
# 가져온 이미지 배치를 그리드 형태로 만듭니다.
# 만들어진 이미지 그리드를 시각화합니다.

Model Train
--------------

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    # 임시 디렉토리 생성 (학습 중 가장 성능이 좋은 모델을 여기에 저장)
        # 현재 모델의 초기 상태를 저장 (기준점)
        # 에폭 반복
            # 각 에폭은 훈련(Train)과 검증(Validation) 두 단계로 구성됨
                # 현재 단계의 데이터셋을 반복
                    # 이전 단계의 gradient 초기화
                    # 순전파 수행
                    # 학습 단계일 때만 gradient 기록
                        # 학습 단계일 경우 역전파 및 최적화 수행
                    # 손실과 정확도 누적 계산
                # 학습 단계일 경우 스케줄러로 learning rate 갱신
                # epoch의 평균 손실과 정확도 계산
                # 검증 단계에서 정확도가 최고 기록이면 모델 저장
        # 전체 학습 시간 계산 및 출력
        # 가장 정확도가 높았던 모델의 가중치 불러오기

모델 예측 시각화

In [ ]:
def visualize_model(model, num_images=6):

## 방법 1. pre-trained  ConvNet 신경망 전체를 미세조정(finetuning)

- 미리 학습된 모델을 불러온 후 마지막의 완전 연결층 만을 새로 작성하고 **전체 parameter** 를 fine tuning

In [ ]:
# 사전 학습된 ResNet-18 모델을 불러옴 (ImageNet 학습 가중치 사용)
# ResNet의 마지막 fully-connected 층(fc)의 입력 feature 수 추출
# 출력 클래스 수를 2개로 설정
# 모델을 device로 이동
# 손실 함수 설정
# 옵티마이저 설정 (SGD: 확률적 경사하강법, 학습률 0.001, 모멘텀 0.9)
# 학습률 스케줄러 설정: 7 에폭마다 학습률을 0.1배씩 감소

In [ ]:
# 학습 함수(train_model)를 호출하여 모델 학습 시작

# 학습된 모델(model_ft)을 시각화하여 예측 결과를 확인
 - 검증 데이터셋에서 일부 이미지를 가져와 모델이 예측한 클래스 이름과 함께 이미지 출력
 -  총 6개의 이미지를 보여줌 (기본값 num_images=6)

## 방법 2 : Pre-trained ConvNet을 weight 고정된 특징 추출기로 사용

- 미리 학습된 모델을 불러온 후 마지막의 **완전 연결층 만을 새로 작성**하되 마지막 계층을 제외한 **신경망의 모든 부분을 고정** (``requires_grad = False`` 로 설정)하여 ``backward()`` 중에 gradient가 계산되지 않도록 함

In [ ]:
# 사전 학습된 ResNet-18 모델 로드 (ImageNet 가중치 사용)
# 모든 파라미터의 requires_grad 값을 False로 설정하여 고정시킴
# → 기존 가중치는 학습되지 않음 (feature extractor로만 사용)
# 새로 정의할 fully-connected 레이어(fc)의 입력 특징 수 추출
# 출력층(fc)을 이진 분류용으로 교체 (출력 클래스 수 = 2)
# 새로 생성한 fc 레이어의 파라미터는 기본적으로 requires_grad=True → 학습 가능
# 손실 함수 설정
# 옵티마이저 설정:
# 이전과 달리 model_conv 전체가 아닌 fc 레이어의 파라미터만 최적화 대상
# 즉, 출력층만 학습되고 나머지 레이어는 고정됨
# 학습률 스케줄러 설정: 7 에폭마다 학습률을 0.1배씩 감소

### 훈련 및 평가

In [ ]:
# 학습된 모델(model_conv)을 이용해 검증 데이터셋에서 예측 결과를 시각화
# 인터랙티브 모드를 끄고 명시적으로 plt.show() 호출할 때까지 기다리도록 설정
# 누적된 그래프/이미지 시각화를 화면에 출력

### 사용자 정의 이미지에 대한 추론
훈련된 모델을 사용하여 사용자 정의 이미지에 대한 예측을 하고 예측된 클래스 레이블을 이미지와 함께 시각화합니다

In [ ]:
# model = model_conv
# 이미지 열기
# 검증 데이터에 사용된 변환(transform) 적용
# 배치 차원을 추가 (모델 입력 형태: [1, C, H, W])
# 추론 실행 (gradient 비활성화 → 메모리, 속도 효율적)
# 시각화: 2x2 서브플롯 중 첫 번째 칸에 이미지 출력

In [ ]:
# 모델을 평가 모드로 설정
        # 출력값 중 최대값을 가진 인덱스를 예측값으로 선택
        # 예측값을 predictions 리스트에 추가
        # 실제 레이블을 labels 리스트에 추가
    # 예측값과 실제 레이블이 일치하는 인덱스를 찾음
    # 예측값과 실제 레이블이 불일치하는 인덱스를 찾음
# 정확도를 계산하여 출력